In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [12, 6]

data_path = "../../data/origineel"

if not Path(data_path).is_dir():
    raise Warning("Data path does not exist")
    
# Lijst van .csv-bestanden
data_file_paths = [p for p in Path(data_path).iterdir() if p.suffix==".csv"]

# Lees de drie bestanden van het circuit as pandas DataFrame
cableconfig = pd.read_csv(data_file_paths[0], sep=";")
partialdischarges = pd.read_csv(data_file_paths[1], sep=";")
warning = pd.read_csv(data_file_paths[2], sep=";")

print("hallo")

hallo


In [2]:
# Namen van de kolommen
time_col, location_col, charge_col = partialdischarges.columns

# Check that each row either has location and charge undefinied, or both defined:
for i in np.arange(len(partialdischarges))[np.isnan(partialdischarges[location_col]) != np.isnan(partialdischarges[charge_col])]:
    raise Warning(f"Row {i} has missing values")

# Lijst met bools die aangeeft of er op dat tijdstip een PD was
pd_occured = ~np.isnan(partialdischarges[location_col])

# De drie kolommen; alleen op tijdstippen met PD
locations = partialdischarges[location_col][pd_occured]
charges = partialdischarges[charge_col][pd_occured]
times = partialdischarges[time_col][pd_occured].apply(datetime.datetime.fromisoformat)

In [9]:
REL_DENSITY_HIGH_BUCKET = 2
NR_BUCKETS_LOC = 5000
NR_BUCKETS_TIME = 1
CLUSTER_RANGE = 10
MINIMUM_CLUSTER_SIZE = 2

In [10]:
def fill_buckets(locations, nr_buckets, min_loc, max_loc):
    buckets = [0] * nr_buckets
    for x in locations:
        buckets[math.floor(nr_buckets*(x-min_loc)/(max_loc-min_loc))] +=1
    return buckets

def hi_dens_buckets(buckets, avg_dens, nr_buckets, rel_dens_hi):
    hi_buckets = []
    for i in range(nr_buckets):
        if(buckets[i]>=rel_dens_hi*avg_dens):
            hi_buckets.append(i)
    return hi_buckets

def connected_points(index, hi_buckets, cluster_range):
    i = index + 1
    while(i+1<len(hi_buckets)and hi_buckets[i+1]-hi_buckets[i]<cluster_range):
        i +=1
    return i-1

            
def find_clusters(hi_buckets, cluster_range, min_cluster_size, min_loc, max_loc):
    left = 0
    while left < len(hi_buckets) :
        right = connected_points(left, hi_buckets, cluster_range)
        if(right-left>=min_cluster_size):
                        
            print("[%s , %s]\n" % (hi_buckets[left], hi_buckets[right]))
        left = right+1


In [11]:
locations_sorted = sorted(locations)
buckets = fill_buckets(locations_sorted, NR_BUCKETS_LOC, 0, 3000)
hi_buckets = hi_dens_buckets(buckets, len(locations)/3000, NR_BUCKETS_LOC, REL_DENSITY_HIGH_BUCKET)
find_clusters(hi_buckets, CLUSTER_RANGE, MINIMUM_CLUSTER_SIZE, 0, 3000)

[622 , 1320]

[1323 , 1385]

[1388 , 1447]

[1450 , 2206]

